In [33]:
from ml_ai1 import *

data = np.loadtxt("classification_train.csv", delimiter=',', skiprows=1)
X_train = data[:, 2:]
y_train = data[:, 1]
m, n = X_train.shape
ctgrs = np.unique(y_train) # np.unique() to get categories
q = len(ctgrs)

# Not doing normalisation so commenting it
# mean = np.mean(X_train, axis=0).reshape(1, n)
# ptp = np.max(X_train, axis=0) - np.min(X_train, axis = 0)
# X_train = (X_train - mean)/ptp

data = np.loadtxt("classification_train.csv", delimiter=',', skiprows=1)
X_train = data[:, 2:]
y_train = data[:, 1]

m, n = X_train.shape
ctgrs = np.unique(y_train) # np.unique() to get categories
q = len(ctgrs)


while True:
    l = input("Number of layers: ")
    if l.isdigit() and int(l) > 0:
        break
    else:
        print(f"Positive integer expected...")
        continue
n_l = int(l)


iters = iterations()
learn_rate = alpha()

activation_layer = Activation_ReLU()
activation_last_layer = Activation_Softmax()
activation_loss = Actvn_Softmax_Loss_CategoricalCrossentropy()
optimiser = SGD(learn_rate)

n_inputs = int(X_train.shape[1])


layer_lst = []

for i in range(n_l):
    n_neurons = neurons()
    if i == 0:
        layer = Gen_layer(n_inputs, n_neurons)
        layer_lst.append(layer)

    if 0 < i <= n_l-1:
        layer1 =Gen_layer(layer_lst[-1].n_neurons, n_neurons)
        layer_lst.append(layer1)

l1 = len(layer_lst)
print(f"Number of layers: {l1} | alpha: {learn_rate}")

for i in range(l1): 
    print(f"Layer {i + 1} : {layer_lst[i].n_neurons} neurons")

for iter in range(iters):
    activations = []
    for i in range(n_l):
        if 0<= i < n_l-1:
            activations.append(Activation_ReLU())
        if i == n_l -1:
            activations.append(Activation_Softmax())
    l = len(layer_lst)
    for j in range(l):
        if j == 0:
            layer_lst[j].forward(X_train)
            activations[j].forward(layer_lst[j].output)

        if 0 < j < l -1:
            layer_lst[j].forward(activations[j-1].output)
            activations[j].forward(layer_lst[j].output)


        if j == l - 1:
            layer_lst[j].forward(activations[j-1].output)
            activations[j].forward(layer_lst[j].output)
        
    # Loss
    loss = activation_loss.forward(layer_lst[l-1].output, y_train)

    # Accuracy
    predictions = np.argmax(activation_loss.output, axis = 1)
    if len(y_train.shape) == 2:
        y_train = np.argmax(y_train, axis = 1)
    accuracy = np.mean(predictions==y_train)

    if iter % 100 == 0:
        print(f"Iteration no. {iter}: " + f"loss {loss:0.4e} " + f"acc: {accuracy:.3f}")
    
    # Last iterations to check variation
    if (iters - 10) <= iter <= (iters - 1):
        print(f"Iteration no. {iter}: " + f"loss {loss:0.4e} " + f"acc: {accuracy:.3f}")  

    lst_der_inputs = []
    activation_loss.backward(activation_loss.output, y_train)

    for k in range(l):
        if k == 0:
            layer_lst[l-1].backward(activation_loss.dinputs)
            der = activations[l-2].backward(layer_lst[l-1].dinputs)
            lst_der_inputs.append(der)
        if 0 < k < l - 1:
                layer_lst[l-1-k].backward(activations[l-1-k].dinputs)
                der = activations[l-2-k].backward(layer_lst[l-1-k].dinputs)
                lst_der_inputs.append(der)
        if (k == l-1):
            layer_lst[0].backward(activations[0].dinputs)
    for a in range(l):
        optimiser.update_p(layer_lst[a])

Number of layers: 2 | alpha: 0.001
Layer 1 : 784 neurons
Layer 2 : 10 neurons
Iteration no. 0: loss 1.4149e+01 acc: 0.120
Iteration no. 100: loss 2.3991e+00 acc: 0.739
Iteration no. 200: loss 1.8839e+00 acc: 0.754
Iteration no. 290: loss 1.6180e+00 acc: 0.764
Iteration no. 291: loss 1.6121e+00 acc: 0.757
Iteration no. 292: loss 1.6132e+00 acc: 0.764
Iteration no. 293: loss 1.6075e+00 acc: 0.757
Iteration no. 294: loss 1.6085e+00 acc: 0.765
Iteration no. 295: loss 1.6029e+00 acc: 0.757
Iteration no. 296: loss 1.6037e+00 acc: 0.765
Iteration no. 297: loss 1.5983e+00 acc: 0.758
Iteration no. 298: loss 1.5990e+00 acc: 0.766
Iteration no. 299: loss 1.5938e+00 acc: 0.758


In [30]:
import pandas as pd
test_data = np.loadtxt("classification_test.csv", delimiter=',', skiprows=1)
X_test = test_data[:, 1:]

l = len(layer_lst)

for j in range(l):
    if j == 0:
        layer_lst[j].forward(X_test)
        activation_layer.forward(layer_lst[j].output)

    if 0 < j < l -1:
        layer_lst[j].forward(activation_layer.output)
        activation_layer.forward(layer_lst[j].output)

    if j == l - 1:
        layer_lst[j].forward(activation_layer.output)
        activation_last_layer.forward(layer_lst[j].output)

df = pd.DataFrame(test_data[:, 0])
df['Output'] = np.argmax(activation_last_layer.output, axis=1)
df.columns = ['ids', 'Output']
print(df)

# print(f"Class: {np.argmax(activation_last_layer.output, axis=1)}")

          ids  Output
0     25672.0       0
1     59964.0       1
2     66230.0       2
3     50801.0       0
4     83307.0       3
...       ...     ...
9995  71009.0       0
9996  67472.0       4
9997  99226.0       8
9998  58519.0       2
9999  80458.0       2

[10000 rows x 2 columns]


In [31]:
df.to_csv("submission_neural_n_layer.csv")

In [19]:
print(f"Class_test: {np.argmax(activation_last_layer.output, axis = 1)}")

Class_test: [6 1 4 ... 8 2 6]


# Rough

In [6]:
a = np.array([[1, 2, 3],
              [3, 5, 6],
              [4, 3, 7]])
np.argmax(a, axis = 0)

array([2, 1, 2], dtype=int64)